<a href="https://colab.research.google.com/github/MarianoChic09/Agentes-Inteligentes/blob/main/DL_Obligatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![AIRBNB](https://www.stevenridercpa.au/wp-content/uploads/2022/09/airbnb-tax.jpeg)

# Obligatorio Deep Learning

# Obligatorio de Deep Learning
## Semestre 2 - 2023
-------

## Problema

Se presenta un dataset que contiene información de alojamientos publicados en AirBnB con sus respectivos precios. El tamaño del dataset de train es de 1.5 Gb aproximadamente, y 0.5 Gb el de test. Este cuenta con 84 variables predictoras que se podrán utilizar como consideren adecuado.

El objetivo es asignar el precio correcto a los alojamientos listados.

Además del dataset se les provee esta notebook conteniendo el script de carga de datos y un modelo baseline que corresponde a una arquitectura feed forward.

------

## Consigna

### A) <u>Participación en Competencia Kaggle</u>:
El objetivo de este punto es participar en la competencia de Kaggle y obtener como mínimo un Mean Absolute Error inferior a 70 puntos. [->Link a la competencia<-](https://www.kaggle.com/t/69c648e3aa214d1f812bf2314c8d4ffa).

### B) <u>Utilización de Grid Search (o equivalente)</u>:
Para cumplir con la busqueda de modelos óptimos se debe realizar un grid search lo más abarcativo y metódico posible. Recomendamos enfáticamente [Weights and Biases](https://wandb.ai/site)

### C) <u>Se debe a su vez investigar e implementar las siguientes técnicas</u>:
#### 1. [Batch Normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/)
#### 2. [Gradient Normalization y/o Gradient Clipping](https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/)


Además como en todas las tareas se evaluará la prolijidad de la entrega, el preprocesamiento de datos, visualizaciones y exploración de técnicas alternativas.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install langdetect
!pip install tqdm
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


# Installing cudf

In [4]:
!nvidia-smi

Fri Nov 24 12:06:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 4.17 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.5 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Found existing installation: cupy-cuda11x 11.0.0
Uninstalling cupy-cuda11x-11.0.0:
  Successfully uninstalled cupy-cuda11x-11.0.0
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint 60C317803A41BA51845E371A1E9377A2BA9EF27F
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/rep

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 40.0.1
Uninstalling cryptography-40.0.1:
  Successfully uninstalled cryptography-40.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.3/446.3 kB 6.3 MB/s eta 0:00:00
Installing RAPIDS Stable 23.10
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-

# Obligatorio Deep Learning
## 1. Setup
### 1.1 Imports

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Datasets

/content/drive/MyDrive/Colab Notebooks/Datasets


In [4]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py



In [5]:
import cudf

# Path to your CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/public_train_data.csv'

# Read the CSV file using dask_cudf
df = cudf.read_csv(file_path)

In [6]:
df.head(2)

,id,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,...,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,Price
0,0,2017-05-12,Grand Loft in the heart of historic Antwerp,Best location for visiting Antwerp!! Beautiful...,Welcome in Antwerp!! The loft is situated on t...,Best location for visiting Antwerp!! Beautiful...,none,<NA>,<NA>,<NA>,...,10.0,9.0,<NA>,<NA>,strict,2.0,2.6,"51.21938762207894, 4.4034442505151885","Host Has Profile Pic,Instant Bookable",159.0
1,1,2017-05-03,"CHARMING, CLEAN & COZY BUNGALOW!",Very centrally located and less than 15 min fr...,"Well lit, private entrance with small patio.",Very centrally located and less than 15 min fr...,none,"Quiet. Pretty tree lined streets, safe area.",Has dining table and high back desk chair.,"Uber, bus line and metro link is less than 5 m...",...,<NA>,<NA>,<NA>,"City of Los Angeles, CA",flexible,1.0,<NA>,"34.1892692286356, -118.41993491931177","Host Has Profile Pic,Is Location Exact",49.0


In [7]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 326287 entries, 0 to 326286
Data columns (total 85 columns):
 #   Column                          Non-Null Count   Dtype
---  ------                          --------------   -----
 0   id                              326287 non-null  int64
 1   Last Scraped                    326286 non-null  object
 2   Name                            326018 non-null  object
 3   Summary                         315651 non-null  object
 4   Space                           228792 non-null  object
 5   Description                     326188 non-null  object
 6   Experiences Offered             326287 non-null  object
 7   Neighborhood Overview           192513 non-null  object
 8   Notes                           130729 non-null  object
 9   Transit                         200649 non-null  object
 10  Access                          177108 non-null  object
 11  Interaction                     169193 non-null  object
 12  House Rules                    

### 3.4 Estadísticas descriptivas

In [8]:
df.describe()

/usr/local/lib/python3.10/site-packages/cudf/core/dataframe.py:5062: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,id,Host ID,Host Response Rate,Host Listings Count,Host Total Listings Count,Latitude,Longitude,Accommodates,Bathrooms,Bedrooms,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Calculated host listings count,Reviews per Month,Price
count,326287.000000,3.262870e+05,250845.000000,325971.000000,325970.000000,326287.000000,326287.000000,326244.000000,325300.000000,325873.000000,...,243160.000000,242584.000000,242732.000000,242378.000000,242710.000000,242423.000000,242347.000000,325689.000000,246983.000000,326287.000000
mean,163143.000000,3.236757e+07,93.408264,9.586000,9.586026,38.042816,-15.323924,3.270764,1.239482,1.358072,...,92.880063,9.524713,9.326067,9.691416,9.708253,9.468215,9.321031,6.881531,1.486211,138.229041
std,94191.087979,3.174572e+07,17.536835,57.399711,57.399797,22.910029,70.101677,2.037446,0.574784,0.921763,...,8.569521,0.855361,1.038858,0.731702,0.723143,0.805116,0.906478,42.025986,1.752082,149.790527
min,0.000000,1.900000e+01,0.000000,0.000000,0.000000,-38.224427,-123.218712,1.000000,0.000000,0.000000,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.010000,0.000000
25%,81571.500000,6.869780e+06,98.000000,1.000000,1.000000,38.923154,-73.968081,2.000000,1.000000,1.000000,...,90.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.320000,55.000000
50%,163143.000000,2.186737e+07,100.000000,1.000000,1.000000,42.304549,0.090277,2.000000,1.000000,1.000000,...,95.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,0.890000,90.000000
75%,244714.500000,4.799166e+07,100.000000,3.000000,3.000000,50.863658,12.342749,4.000000,1.000000,2.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,2.040000,150.000000
max,326286.000000,1.350885e+08,100.000000,1114.000000,1114.000000,55.994889,153.637837,18.000000,8.000000,96.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,752.000000,223.000000,999.000000


In [9]:
df.columns

Index(['id', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description',
       'Experiences Offered', 'Neighborhood Overview', 'Notes', 'Transit',
       'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url',
       'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name',
       'Host Since', 'Host Location', 'Host About', 'Host Response Time',
       'Host Response Rate', 'Host Acceptance Rate', 'Host Thumbnail Url',
       'Host Picture Url', 'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Security Deposit',
       'Cleaning Fee', 'Guests Included', 'Extra Peop

In [10]:
df.shape

(326287, 85)

In [11]:
import cudf
import cuml
from cuml.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
# from cudf.core.subword_tokenizer import SubwordTokenizer
# import dask_cudf
# import dask
import numpy as np

# Assuming cuDF tokenizer setup and other relevant imports are already done

# Define the threshold for unique values to differentiate categorical and textual data
unique_value_threshold = 20

# Load your data into a cuDF DataFrame
# df = cudf.read_csv('your_data.csv')  # Replace with your data source

# Separate features and target
df_train = df.drop('Price', axis=1)
y = df['Price']

# Identify numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Initialize lists for categorical and textual columns
categorical_cols = []
text_cols = []

# Iterate over columns and classify them as categorical or textual based on unique values
for col in df.columns:
    if col not in numerical_cols:
        unique_values = df[col].nunique(dropna=False)
        if unique_values < unique_value_threshold:
            categorical_cols.append(col)
        else:
            text_cols.append(col)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=0)

# Define important text columns
important_text_cols = ['Name', 'Summary', 'Description', 'Neighborhood Overview', 'Transit', 'Access', 'Interaction', 'House Rules']
text_cols_important = [column for column in text_cols if column in important_text_cols]


In [12]:
# Now create the cuDF DataFrame
X_text_train = cudf.DataFrame(X_train[text_cols_important])
X_text_test = cudf.DataFrame(X_test[text_cols_important])

In [13]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# List of top 10 languages (modify as needed)
languages = ['english', 'spanish', 'chinese', 'arabic', 'portuguese', 'indonesian', 'french', 'russian', 'german']

# Create a combined set of stopwords from these languages
combined_stopwords = set()
for lang in languages:
    combined_stopwords.update(stopwords.words(lang))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
import cudf
import re
from bs4 import BeautifulSoup
import numpy as np

# Define the batch size
batch_size = 1000  # Adjust this based on your GPU memory

# Function to clean a batch of text
def clean_batch(batch):
    # Remove HTML tags
    batch = batch.str.replace('<.*?>', '', regex=True)

    # Correct encoding issues and remove special characters
    batch = batch.str.replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">")
    batch = batch.str.replace(r'[^a-zA-Z0-9.,!?/:;\"\'\s]', '', regex=True)

    # Replace multiple spaces with a single space
    batch = batch.str.replace(r'\s+', ' ', regex=True).str.strip()

    return batch

# Function to process text data in batches
def process_in_batches(df, column_name):
    num_rows = len(df)
    num_batches = np.ceil(num_rows / batch_size).astype(int)

    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min(batch_start + batch_size, num_rows)
        batch = df[column_name].iloc[batch_start:batch_end]
        cleaned_batch = clean_batch(batch)
        df[column_name].iloc[batch_start:batch_end] = cleaned_batch

    return df

# Apply cleaning to each important text column
for col in important_text_cols:
    X_text_train = process_in_batches(X_text_train, col)
    X_text_test = process_in_batches(X_text_test, col)

In [15]:
X_text_train.head(5)

,Name,Summary,Description,Neighborhood Overview,Transit,Access,Interaction,House Rules
67291,Modern flat in a greenquiet area,Our two bedroom flat is part of a new building...,Our two bedroom flat is part of a new building...,Quiet and friendly families and young couples ...,17 minutes via foot away from Hackbridge train...,<NA>,We try to be available via emailtelephonein pe...,Please ensure you remove your shoes once you a...
41204,3 blocks from SOCO Living,"Tree Studio in Travis Heights, is 3 blocks fro...","Tree Studio in Travis Heights, is 3 blocks fro...",Despite its proximity to South Congress and th...,There's a major bus route 3 blocks from the st...,"Refrigerator, Bathroom, Internet, Record Playe...",You will have the studio all to yourself but i...,Key exchange will be done on day of arrival an...
282576,Piazza Fiume sognando la Dolce Vita,Accogliente e confortevole appartamento recent...,Accogliente e confortevole appartamento recent...,L' appartamento situato nel quartiere Salario ...,La zona dove si trova l'appartamento ben colle...,Gli ospiti possono usufruire dell'intero appar...,Sar a disposizione degli ospiti per qualsiasi ...,Facendo parte di un condominio non possibile f...
2822,FamilyFriendly Modern Ranch in Great SE Location!,Welcome to this SE Portland familyfriendly hou...,Welcome to this SE Portland familyfriendly hou...,"This is a great neighborhood. It is quiet, cle...",It is really easy to get around Portland from ...,The washer and dryer.,<NA>,"The home office, garage and tiny house in the ..."
257477,Lovely openspace flat 45m2 near Bastille,"Welcome to a super well located modern flat, p...","Welcome to a super well located modern flat, p...",<NA>,<NA>,<NA>,<NA>,<NA>


In [16]:
# Concatenate the text columns
for df in [X_text_train, X_text_test]:
    # Convert all columns to string type
    for col in important_text_cols:
        df[col] = df[col].astype('str')

    # Initialize the combined_text column
    df['combined_text'] = ''

    # Concatenate each column to the combined_text
    for col in important_text_cols:
        df['combined_text'] = df['combined_text'] + df[col] + ' '

    # Strip any trailing whitespaces
    df['combined_text'] = df['combined_text'].str.strip()


In [17]:
X_text_train['combined_text']

67291                                                  <NA>
41204     3 blocks from SOCO Living Tree Studio in Travi...
282576    Piazza Fiume sognando la Dolce Vita Accoglient...
2822                                                   <NA>
257477                                                 <NA>
                                ...                        
122579                                                 <NA>
304137                                                 <NA>
152315                                                 <NA>
117952                                                 <NA>
305711                                                 <NA>
Name: combined_text, Length: 261029, dtype: object

In [18]:
# Load BERT tokenizer and model
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
seed = 42

from transformers import DistilBertTokenizer, TFDistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

# Rest of your code for processing...


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [19]:
from transformers import BertTokenizer
import tensorflow as tf

max_length = 300  # Reduce the sequence length if needed


def batch_encode_cudf(cudf_series, batch_size=512):
    input_ids = []
    attention_masks = []

    # Convert cuDF Series to Pandas Series for compatibility
    texts = cudf_series.to_pandas()

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    # Optionally, convert results back to cuDF DataFrames for further GPU processing
    input_ids = cudf.DataFrame.from_pandas(pd.DataFrame(input_ids.numpy()))
    attention_masks = cudf.DataFrame.from_pandas(pd.DataFrame(attention_masks.numpy()))

    return input_ids, attention_masks

In [20]:
def batch_encode(texts, batch_size=512):
    input_ids = []
    attention_masks = []

    # Convert cuDF Series to Pandas Series and handle null values
    if isinstance(texts, cudf.Series):
        texts = texts.to_pandas().fillna("")

    # Ensure all data are strings
    texts = texts.astype(str)

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)
    return input_ids, attention_masks

# Example usage
train_input_ids, train_attention_masks = batch_encode(X_text_train['combined_text'])
test_input_ids, test_attention_masks = batch_encode(X_text_test['combined_text'])



In [21]:
def create_dataset(input_ids, attention_masks, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_masks
        },
        labels
    ))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Scaling the labels
price_scaler = StandardScaler()
price_scaler.fit(y_train.to_numpy().reshape(-1, 1))
train_labels = price_scaler.transform(y_train.to_numpy().reshape(-1, 1))#.reshape(-1, 1))
test_labels = price_scaler.transform(y_test.to_numpy().reshape(-1, 1))#.reshape(-1, 1))

# labels = [...]  # Your label data
train_dataset = create_dataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = create_dataset(test_input_ids, test_attention_masks, test_labels)

## Train model text only

In [ ]:
# Define the model
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

def build_model():
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    bert_output = bert_model([input_ids, attention_mask])
    cls_token_output = bert_output.last_hidden_state[:, 0, :]
    output = tf.keras.layers.Dense(1)(cls_token_output)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-8),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])
    return model

model = build_model()

epochs=5

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs
)
model.save_weights(f'trained_bert_{epochs}.h5')
# Loading model


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
model.save_weights(f'trained_bert_2ymedia.h5')


In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/trained_bert.h5')


### Kaggle test data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Assuming 'basic_cleaning' and 'batch_encode' functions are already defined

# Load the test data
file_path2 = './obligatorio_DL/private_data_to_predict.csv'
kaggle_test = cudf.read_csv(file_path2)

# Apply basic cleaning


# Apply cleaning to each important text column
for col in important_text_cols:
    kaggle_test = process_in_batches(kaggle_test, col)




In [ ]:
# kaggle_test = data_for_kaggle.applymap(basic_cleaning)
kaggle_test['combined_text'] = ''

for col in important_text_cols:
    kaggle_test[col] = kaggle_test[col].astype('str')
    kaggle_test['combined_text'] = kaggle_test['combined_text'] + kaggle_test[col] + ' '
    kaggle_test['combined_text'] = kaggle_test['combined_text'].str.strip()

kaggle_input_ids, kaggle_attention_masks = batch_encode(kaggle_test['combined_text'])

# Make predictions


In [ ]:
predictions = model.predict(x=[kaggle_input_ids, kaggle_attention_masks])

# Rescale predictions
y_pred = price_scaler.inverse_transform(predictions)



array([[207.5733  ],
       [109.61352 ],
       [118.88889 ],
       ...,
       [118.888885],
       [118.888885],
       [118.888885]], dtype=float32)

In [ ]:
# Prepare output
test_ids = kaggle_test['id']
output = np.hstack((test_ids.to_numpy().reshape(-1, 1), y_pred))
df = pd.DataFrame(output, columns=['id', 'expected'])

# Handle any NaNs
df['expected'] = df['expected'].fillna(0)

# Save to CSV
df.to_csv("output_to_submit.csv", index=False)

## Incorporating Numerical and Categorical data

In [22]:
numerical_cols.remove('Price')

In [23]:
# Scaling numerical columns
import cupy as cp

scaler = cuml.preprocessing.StandardScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_cols])
X_test_numerical = scaler.transform(X_test[numerical_cols])

# Encoding categorical columns
encoder = cuml.preprocessing.OneHotEncoder(handle_unknown='ignore')
X_train_categorical = encoder.fit_transform(X_train[categorical_cols])
X_test_categorical = encoder.transform(X_test[categorical_cols])

# Handling missing values (if needed)
# You can use cuDF's fillna() method or cuml's SimpleImputer
# Example for numerical columns:
X_train_numerical = X_train_numerical.fillna(X_train_numerical.mean())
X_test_numerical = X_test_numerical.fillna(X_test_numerical.mean())

# Convert to Pandas DataFrame
X_train_numerical_pd = X_train_numerical.to_pandas()
# X_train_categorical_pd = X_train_categorical.to_pandas()
X_test_numerical_pd = X_test_numerical.to_pandas()
# X_test_categorical_pd = X_test_categorical.to_pandas()
# Converting coo_matrix to dense format and then to Pandas DataFrame
# X_train_categorical_pd = pd.DataFrame(X_train_categorical.toarray())
# X_test_categorical_pd = pd.DataFrame(X_test_categorical.toarray())
# Convert the cuML OneHotEncoder output (sparse matrix) to a dense CuPy array
# Convert the CuPy array to a NumPy array
# Convert the cuML OneHotEncoder output (sparse matrix) to a dense CuPy array
X_train_categorical_cupy = cp.asarray(X_train_categorical.toarray())
X_test_categorical_cupy = cp.asarray(X_test_categorical.toarray())

X_train_categorical_np = X_train_categorical_cupy.get()
X_test_categorical_np = X_test_categorical_cupy.get()

# Convert the NumPy array to a TensorFlow tensor
X_train_categorical_tf = tf.convert_to_tensor(X_train_categorical_np)
X_test_categorical_tf = tf.convert_to_tensor(X_test_categorical_np)


# Convert to TensorFlow tensors
X_train_numerical_tf = tf.convert_to_tensor(X_train_numerical_pd.values, dtype=tf.float32)
# X_train_categorical_tf = [tf.convert_to_tensor(X_train_categorical_pd[col].values, dtype=tf.int32) for col in X_train_categorical_pd.columns]
X_test_numerical_tf = tf.convert_to_tensor(X_test_numerical_pd.values, dtype=tf.float32)
# X_test_categorical_tf = [tf.convert_to_tensor(X_test_categorical_pd[col].values, dtype=tf.int32) for col in X_test_categorical_pd.columns]


In [24]:
# Retrieve new column names
encoded_column_names = encoder.get_feature_names(categorical_cols)

# Print the new column names
print("Encoded Column Names:", encoded_column_names)
print("length of encoded column names:",len(encoded_column_names))

Encoded Column Names: ['Experiences Offered_1' 'Experiences Offered_business'
 'Experiences Offered_family' 'Experiences Offered_none'
 'Experiences Offered_romantic' 'Experiences Offered_social'
 'Host Response Time_-122.41000020536359'
 'Host Response Time_a few days or more' 'Host Response Time_within a day'
 'Host Response Time_within a few hours'
 'Host Response Time_within an hour' 'Host Response Time_None'
 'Room Type_9' 'Room Type_Entire home/apt' 'Room Type_Private room'
 'Room Type_Shared room' 'Bed Type_9' 'Bed Type_Airbed' 'Bed Type_Couch'
 'Bed Type_Futon' 'Bed Type_Pull-out Sofa' 'Bed Type_Real Bed'
 'Has Availability_0.21' 'Has Availability_t' 'Has Availability_None'
 'Cancellation Policy_flexible' 'Cancellation Policy_flexible_new'
 'Cancellation Policy_moderate' 'Cancellation Policy_moderate_new'
 'Cancellation Policy_no_refunds' 'Cancellation Policy_strict'
 'Cancellation Policy_strict_new' 'Cancellation Policy_super_strict_30'
 'Cancellation Policy_super_strict_30_ne

In [25]:
# Assuming the following variables are already defined:
# X_train_numerical_tf, X_test_numerical_tf, X_train_categorical_tf, X_test_categorical_tf
# train_input_ids, test_input_ids, train_attention_masks, test_attention_masks

print("Shapes of Train and Test Data Components:")
print("-----------------------------------------")

# Print shapes of numerical data
print("X_train_numerical_tf shape:", X_train_numerical_tf.shape)
print("X_test_numerical_tf shape:", X_test_numerical_tf.shape)

# Print shapes of categorical data
print("X_train_categorical_tf shape:", X_train_categorical_tf.shape)
print("X_test_categorical_tf shape:", X_test_categorical_tf.shape)

# Print shapes of tokenized inputs
print("train_input_ids shape:", train_input_ids.shape)
print("test_input_ids shape:", test_input_ids.shape)
print("train_attention_masks shape:", train_attention_masks.shape)
print("test_attention_masks shape:", test_attention_masks.shape)

Shapes of Train and Test Data Components:
-----------------------------------------
X_train_numerical_tf shape: (261029, 32)
X_test_numerical_tf shape: (65258, 32)
X_train_categorical_tf shape: (261029, 37)
X_test_categorical_tf shape: (65258, 37)
train_input_ids shape: (261029, 300)
test_input_ids shape: (65258, 300)
train_attention_masks shape: (261029, 300)
test_attention_masks shape: (65258, 300)


In [26]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_filepath = 'model_weights_epoch_{epoch:02d}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=False,
    save_freq='epoch'
)

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

num_numerical_features = len(numerical_cols)
num_categorical_features = len(encoded_column_names)

input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
# bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]
bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]

bert_dense = tf.keras.layers.Dense(64, activation='relu')(bert_output)

# Numerical Input
numerical_input = tf.keras.layers.Input(shape=(num_numerical_features,), name='numerical_data')
numerical_dense = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01))(numerical_input)

# Categorical Input
categorical_input = tf.keras.layers.Input(shape=(num_categorical_features,), name='categorical_data')
categorical_dense = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01))(categorical_input)


concatenated = tf.keras.layers.concatenate([bert_dense, numerical_dense, categorical_dense])
# Concatenate all inputs
dropout_layer = Dropout(0.5)(concatenated)

# Additional layers as needed
output_layer = tf.keras.layers.Dense(1)(dropout_layer)

# Build model
model = tf.keras.Model(inputs=[input_ids, attention_mask, numerical_input, categorical_input], outputs=output_layer)


In [28]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [34]:
# Import the W&B Python Library and log into W&B
import wandb

wandb.login()

#Creamos un proyecto en WandB a través de su interfaz
project = "obligatorio_dl"
entity = "marian-ai"

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD

# Define the sweep configuration
sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        # Add other hyperparameters here
    }
}

def train():
    with wandb.init() as run:
        config = run.config

        # Callbacks
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
        model_checkpoint_callback = ModelCheckpoint(
            filepath='model_weights_epoch_{epoch:02d}.h5',
            save_weights_only=True,
            save_best_only=False,
            save_freq='epoch'
        )

        # Build and compile your model here using config parameters
        # For example, setting the optimizer
        optimizer = Adam(learning_rate=config.learning_rate)
        # model.compile(optimizer=optimizer, ...)

        # Fit the model
        history = model.fit(
            {
                'input_ids': train_input_ids,
                'attention_mask': train_attention_masks,
                'numerical_data': X_train_numerical_tf,
                'categorical_data': X_train_categorical_tf
            },
            train_labels,
            validation_data=(
                {
                    'input_ids': test_input_ids,
                    'attention_mask': test_attention_masks,
                    'numerical_data': X_test_numerical_tf,
                    'categorical_data': X_test_categorical_tf
                },
                test_labels
            ),
            epochs=10,  # You can also use config.epochs if defined in sweep_config
            batch_size=int(config.batch_size),
            callbacks=[early_stopping, reduce_lr, model_checkpoint_callback]
        )
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project=project)

# Run the sweep agent
wandb.agent(sweep_id, function=train, count=10, project=project)

Create sweep with ID: 24f4bwy8
Sweep URL: https://wandb.ai/marian-ai/obligatorio_dl/sweeps/24f4bwy8


wandb: Agent Starting Run: bechg4rf with config:
wandb: 	batch_size: 256
wandb: 	learning_rate: 0.0005375532911570234


Epoch 1/10


Traceback (most recent call last):
  File "<ipython-input-36-93fce9622464>", line 43, in train
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py", line 60, in quick_execute
    tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
tensorflow.python.framework.errors_impl.ResourceExhaustedError: Graph execution error:

Detected at node model/tf_distil_bert_model_1/distilbert/transformer/layer_._0/ffn/Gelu/Erf defined at (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 973, in _bootstrap

  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner

  File "/usr/lib/python3.10/threading.py", line 953, in run

  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 298, in _run_job

  File "<ipython

In [35]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/model_weights_epoch_05.h5')
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'data_combination': {
            'values': ['numerical', 'num_cat', 'num_cat_text']
        },
        # Add other hyperparameters here
    }
}

def train():
    with wandb.init() as run:
        config = run.config

        # Callbacks
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
        model_checkpoint_callback = ModelCheckpoint(
            filepath='model_weights_epoch_{epoch:02d}.h5',
            save_weights_only=True,
            save_best_only=False,
            save_freq='epoch'
        )

        # Build and compile your model here using config parameters
        # For example, setting the optimizer
        optimizer = Adam(learning_rate=config.learning_rate)
        # model.compile(optimizer=optimizer, ...)

        # Select data based on the chosen combination
        if config.data_combination == 'numerical':
            input_data = {'numerical_data': X_train_numerical_tf}
            validation_data = {'numerical_data': X_test_numerical_tf}
        elif config.data_combination == 'num_cat':
            input_data = {
                'numerical_data': X_train_numerical_tf,
                'categorical_data': X_train_categorical_tf
            }
            validation_data = {
                'numerical_data': X_test_numerical_tf,
                'categorical_data': X_test_categorical_tf
            }
        elif config.data_combination == 'num_cat_text':
            input_data = {
                'input_ids': train_input_ids,
                'attention_mask': train_attention_masks,
                'numerical_data': X_train_numerical_tf,
                'categorical_data': X_train_categorical_tf
            }
            validation_data = {
                'input_ids': test_input_ids,
                'attention_mask': test_attention_masks,
                'numerical_data': X_test_numerical_tf,
                'categorical_data': X_test_categorical_tf
            }

        # Fit the model
        history = model.fit(
            input_data,
            train_labels,
            validation_data=(validation_data, test_labels),
            epochs=10,  # Or config.epochs if defined in sweep_config
            batch_size=int(config.batch_size),
            callbacks=[early_stopping, reduce_lr, model_checkpoint_callback]
        )

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project=project)

# Run the sweep agent
wandb.agent(sweep_id, function=train, count=10, project=project)


Create sweep with ID: 7jauby7v
Sweep URL: https://wandb.ai/marian-ai/obligatorio_dl/sweeps/7jauby7v


wandb: Agent Starting Run: i9t5qbf5 with config:
wandb: 	batch_size: 64
wandb: 	data_combination: numerical
wandb: 	learning_rate: 0.0004288002783228434
wandb: Currently logged in as: mariano-chicatun (marian-ai). Use `wandb login --relogin` to force relogin


Epoch 1/10


Traceback (most recent call last):
  File "<ipython-input-35-30605d4c367c>", line 73, in train
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/tmp/__autograph_generated_filen0ckdp6o.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File

Run i9t5qbf5 errored: ValueError('in user code:\n\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1377, in train_function  *\n        return step_function(self, iterator)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1360, in step_function  **\n        outputs = model.distribute_strategy.run(run_step, args=(data,))\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1349, in run_step  **\n        outputs = model.train_step(data)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1126, in train_step\n        y_pred = self(x, training=True)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler\n        raise e.with_traceback(filtered_tb) from None\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 197, in assert_input_compatibility\n        ra

Epoch 1/10


Traceback (most recent call last):
  File "<ipython-input-35-30605d4c367c>", line 73, in train
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/tmp/__autograph_generated_filen0ckdp6o.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File

Run 84s3i52u errored: ValueError('in user code:\n\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1377, in train_function  *\n        return step_function(self, iterator)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1360, in step_function  **\n        outputs = model.distribute_strategy.run(run_step, args=(data,))\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1349, in run_step  **\n        outputs = model.train_step(data)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1126, in train_step\n        y_pred = self(x, training=True)\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler\n        raise e.with_traceback(filtered_tb) from None\n    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 197, in assert_input_compatibility\n        ra

In [ ]:
batch_size = 128
epochs = 10
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)
import datetime

current_time = datetime.datetime.now()

history = model.fit(
    {
        'input_ids': train_input_ids,
        'attention_mask': train_attention_masks,
        'numerical_data': X_train_numerical_tf,
        'categorical_data': X_train_categorical_tf
    },
    train_labels,
    validation_data=(
        {
            'input_ids': test_input_ids,
            'attention_mask': test_attention_masks,
            'numerical_data': X_test_numerical_tf,
            'categorical_data': X_test_categorical_tf
        },
        test_labels
    ),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping, reduce_lr,model_checkpoint_callback]

)
model.save_weights(f'num_cat_and_trained_bert_{epochs}epochs_{current_time}.h5')



Epoch 1/10
2040/2040 [==============================] - 1869s 916ms/step - loss: 0.5552 - mae: 0.4238 - val_loss: 0.4969 - val_mae: 0.3915 - lr: 0.0010
Epoch 2/10
2040/2040 [==============================] - 1869s 916ms/step - loss: 0.5252 - mae: 0.4146 - val_loss: 0.4651 - val_mae: 0.3787 - lr: 0.0010
Epoch 3/10
2040/2040 [==============================] - 1869s 916ms/step - loss: 0.5720 - mae: 0.4164 - val_loss: 0.4479 - val_mae: 0.3681 - lr: 0.0010
Epoch 4/10
2040/2040 [==============================] - 1869s 916ms/step - loss: 0.4983 - mae: 0.4044 - val_loss: 0.4368 - val_mae: 0.3706 - lr: 0.0010
Epoch 5/10
2040/2040 [==============================] - 1869s 916ms/step - loss: 0.4902 - mae: 0.4011 - val_loss: 0.4318 - val_mae: 0.3729 - lr: 0.0010
Epoch 6/10
2040/2040 [==============================] - ETA: 0s - loss: 0.4827 - mae: 0.3981

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-64faaa3a29d0>", line 11, in <cell line: 11>
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 237, in make_fid
    fid = h5f.create(name, h5f.ACC_TRUNC, fapl=fapl, fcpl=fcpl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 126, in h5py.h5f.create
OSError: [Errno 107] Unable to create f

In [ ]:
model.save_weights(f'num_cat_and_trained_bert_{epochs}epochs.h5')


In [ ]:
import os

# Your code here...

# Command to crash the Colab runtime
os._exit(0)

### Test with kaggle (Num, Categorical and Text)

In [29]:
# model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/num_cat_and_trained_bert_10epochs.h5')
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/model_weights_epoch_05.h5')

In [30]:
import cudf
import cuml
import tensorflow as tf
import numpy as np
import pandas as pd
from cuml.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Load the Kaggle test dataset
file_path2 = './obligatorio_DL/private_data_to_predict.csv'
kaggle_test = cudf.read_csv(file_path2)

# Assuming your important text columns are already defined
important_text_cols = ['Name', 'Summary', 'Description', 'Neighborhood Overview', 'Transit', 'Access', 'Interaction', 'House Rules']
kaggle_test_text = kaggle_test[important_text_cols]

# Apply cleaning and encoding to text data
for col in important_text_cols:
    # Assuming process_in_batches is your batch processing function
    kaggle_test_text = process_in_batches(kaggle_test_text,col) #[col].astype(str)

# Combine the cleaned text columns
kaggle_test_text['combined_text'] = ''
for col in important_text_cols:
    kaggle_test_text['combined_text'] = kaggle_test_text['combined_text'] + kaggle_test_text[col] + ' '
    kaggle_test_text['combined_text'] = kaggle_test_text['combined_text'].str.strip()
# Encode the combined text
kaggle_input_ids, kaggle_attention_masks = batch_encode(kaggle_test_text['combined_text'])

# X_kaggle_numerical = kaggle_test[numerical_cols]
# X_kaggle_categorical = kaggle_test[categorical_cols]

# # Apply transformations
# X_kaggle_numerical_transformed = scaler.transform(X_kaggle_numerical)  # Assuming 'scaler' is your StandardScaler
# X_kaggle_categorical_transformed = encoder.transform(X_kaggle_categorical)  # Assuming 'encoder' is your OneHotEncoder

# # Convert to TensorFlow tensors
# X_kaggle_numerical_tf = tf.convert_to_tensor(X_kaggle_numerical_transformed)
# X_kaggle_categorical_tf = tf.convert_to_tensor(X_kaggle_categorical_transformed)





In [31]:
# Assuming kaggle_test is your DataFrame loaded with the test data for Kaggle

# Preprocess Numerical Columns
# Scale the numerical columns using the same scaler as for the training data
kaggle_test_numerical = scaler.transform(kaggle_test[numerical_cols])
kaggle_test_numerical = kaggle_test_numerical.fillna(kaggle_test_numerical.mean())
kaggle_test_numerical_pd = kaggle_test_numerical.to_pandas()
kaggle_test_numerical_tf = tf.convert_to_tensor(kaggle_test_numerical_pd.values, dtype=tf.float32)

# Categorical
kaggle_test_categorical = encoder.transform(kaggle_test[categorical_cols])
kaggle_test_categorical_cupy = cp.asarray(kaggle_test_categorical.toarray())
kaggle_test_categorical_np = kaggle_test_categorical_cupy.get()
kaggle_test_categorical_tf = tf.convert_to_tensor(kaggle_test_categorical_np)

# Now kaggle_test_numerical_tf and kaggle_test_categorical_tf are ready to be used in your model prediction


In [32]:
# Make predictions
predictions = model.predict({
    'input_ids': kaggle_input_ids,
    'attention_mask': kaggle_attention_masks,
    'numerical_data': kaggle_test_numerical_tf,
    'categorical_data': kaggle_test_categorical_tf
})

# Rescale predictions
y_pred = price_scaler.inverse_transform(predictions)

# Prepare output for submission
test_ids = kaggle_test['id']
output = np.hstack((test_ids.to_numpy().reshape(-1, 1), y_pred))
submission_df = pd.DataFrame(output, columns=['id', 'expected'])
submission_df['expected'] = submission_df['expected'].fillna(0)
submission_df.to_csv("output_to_submit.csv", index=False)

5023/5023 [==============================] - 1646s 327ms/step


# Basura

In [ ]:
def create_dataset(input_ids, attention_masks, numerical_data, categorical_data, labels, batch_size=32):
    # Ensure categorical data is a list of tensors, one for each categorical column
    if not isinstance(categorical_data, list):
        categorical_data = [categorical_data]

    # Create a dictionary for all inputs
    input_dict = {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'numerical_data': numerical_data
    }

    # Add each categorical feature to the input dictionary
    for i, cat_data in enumerate(categorical_data):
        input_dict[f'cat_input_{i}'] = cat_data

    # Create the TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((input_dict, labels))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset
# Assuming categorical_data_train and categorical_data_test are lists of categorical columns
# For each categorical column, convert to a TensorFlow tensor
categorical_data_train = [tf.convert_to_tensor(categorical_data_train[i], dtype=tf.int32) for i in range([num_categorical_cols])]
categorical_data_test = [tf.convert_to_tensor(categorical_data_test[i], dtype=tf.int32) for i in range(num_categorical_cols)]

# Convert numerical data to TensorFlow tensors
numerical_data_train = tf.convert_to_tensor(numerical_data_train, dtype=tf.float32)
numerical_data_test = tf.convert_to_tensor(numerical_data_test, dtype=tf.float32)

# Create the datasets
train_dataset = create_dataset(train_input_ids, train_attention_masks, numerical_data_train, categorical_data_train, train_labels)
test_dataset = create_dataset(test_input_ids, test_attention_masks, numerical_data_test, categorical_data_test, test_labels)


NameError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Normalization
from tensorflow.keras.models import Model

# Define input layers
input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
numerical_input = Input(shape=(numerical_cols,), name='numerical_data')
categorical_inputs = [Input(shape=(1,), name=f'cat_input_{i}') for i in range(categorical_cols)]

# BERT model for text
bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]

# Numerical data processing
numerical_layer = Normalization()(numerical_input)
numerical_output = Dense(64, activation='relu')(numerical_layer)

# Categorical data processing
embedding_layers = [Embedding(input_dim=cat_col_vocab_size, output_dim=embedding_dim)(cat_input) for cat_input, embedding_dim in zip(categorical_inputs, embedding_dims)]
flattened_embeddings = [tf.keras.layers.Flatten()(embedding) for embedding in embedding_layers]

# Concatenate all outputs
concatenated = Concatenate()([bert_output, numerical_output] + flattened_embeddings)
dense_layer = Dense(128, activation='relu')(concatenated)
output = Dense(1)(dense_layer)

# Build and compile the model
model = Model(inputs=[input_ids, attention_mask, numerical_input] + categorical_inputs, outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=5)


TypeError: ignored

In [ ]:
from cudf.core.subword_tokenizer import SubwordTokenizer
from cudf.utils.hash_vocab_utils import hash_vocab

vocab_file = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/bert-multilingual-cased/multi_cased_L-12_H-768_A-12/vocab.txt'
# Assuming you have the BERT multilingual vocabulary file
# vocab_file = 'bert-base-multilingual-uncased-vocab.txt'
hash_vocab(vocab_file, 'voc_hash.txt')


In [ ]:

# Initialize tokenizer
tokenizer = SubwordTokenizer('voc_hash.txt', do_lower_case=True)

# Tokenize function
def tokenize_text_column(df, col_name, max_length=128):
    tokenized_output = tokenizer(
        df[col_name],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokenized_output['input_ids']

# Apply tokenization
for col in text_cols:
    df[col] = tokenize_text_column(df, col)
